# Physics 300 
## Computational Physics I (Fall 2017)
## BPB-248, Tues/Thurs 10:00 - 11:15 am 

|Instructor| Prof. Qiang Zhu|
|--|-------------------------------|
|Email | qiang.zhu@unlv.edu|
|Website|http://www.physics.unlv.edu/~qzhu/|
|Office| BPB 232|
|Office hours | Tues/Thurs 8:30 - 10:00 |

# 22 Global Optmization (III)

## 22.1 Genetic Algorithm

A genetic algorithm is a search heuristic that is inspired by Charles Darwin’s theory of natural evolution. This algorithm reflects the process of natural selection where the fittest individuals are selected for reproduction in order to produce offspring of the next generation.

- 1, Initialization
- 2, Selection
- 3, Genetic operators
- 4, Termination

## 22.2 Application to LJ clusters

In [7]:
import numpy as np

def LJ(r):
    r6 = r**6
    r12 = r6*r6
    return 4*(1/r12 - 1/r6)

def total_energy(positions):
    """
    Calculate the total energy
    input:
    positions: 3*N array which represents the atomic positions
    output
    E: the total energy
    """
    E = 0
    N_atom = int(len(positions)/3)

    #positions = [x0, y0, z0, x1, y1, z1, .....  , xn, yn, zn]
    for i in range(N_atom-1):
        for j in range(i+1, N_atom):
            pos1 = positions[i*3:(i+1)*3]
            pos2 = positions[j*3:(j+1)*3]
            #print('pos1:  ', pos1)
            #print('pos2:  ', pos2)
            dist = np.linalg.norm(pos1-pos2)
            #print(i,j, dist)
            E += LJ(dist)
    return E
            
def init_pos(N, L=5):
    return L*np.random.random_sample((N*3,))

In [24]:
from scipy.optimize import minimize
import numpy as np

def init_population(population_size, N_atom):
    cluster = []
    for i in range(population_size):
        cluster.append(init_pos(N_atom))
    return np.array(cluster)

def local_optimize(cluster):
    fitness = []
    for i, cluster0 in enumerate(cluster):
        res = minimize(total_energy, cluster0, method='CG', tol=1e-3) 
        cluster0 = res.x
        fitness.append(res.fun)
    return cluster, np.array(fitness)
        
def ranking(fitness):
    return np.argsort(fitness)

def mutation(cluster, rank):
    
    id = int(len(rank)*np.random.random()/2)
    cluster0 = cluster[id]
    return cluster0 + kT*np.random.random_sample((len(cluster0),))

def crossover(cluster, rank):
    id1 = int(len(rank)*np.random.random()/2)
    while True:
        id2 = int(len(rank)*np.random.random()/2)
        if id2 != id1:
            break
    frac = np.random.random()
    return cluster[id1]*frac + cluster[id2]*(1-frac)


def GA(generation_num=10, population_num=10, atom_num=10):
    for i in range(generation_num):
        if i == 0:
            cluster = init_population(population_num, atom_num)
            
        cluster, fitness = local_optimize(cluster)
        #print(fitness)
        print('best fitness in generation ', i, ':  ', min(fitness))
        
        rank = ranking(fitness)
        new_cluster = []
        for j in range(population_num):
            if j < int(0.7*population_num):
                new_cluster.append(crossover(cluster, rank))
            else:
                new_cluster.append(crossover(cluster, rank))
        cluster = new_cluster

In [25]:
GA(atom_num=10)

best fitness in generation  0 :   -27.5558630228
best fitness in generation  1 :   -27.5452068387
best fitness in generation  2 :   -26.4904986844
best fitness in generation  3 :   -27.4797388897
best fitness in generation  4 :   -27.446828942
best fitness in generation  5 :   -28.4225318671
best fitness in generation  6 :   -27.5558630165
best fitness in generation  7 :   -27.5558630297
best fitness in generation  8 :   -27.4797389021
best fitness in generation  9 :   -27.5558630026


## 22.3 Homework

Check the following documentation and use the differential evolution method to perform optimization on LJ clusters
https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.differential_evolution.html#scipy.optimize.differential_evolution